In [1]:
#!pip install mne autoreject eeg_positions pyriemann

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from os import getcwd, listdir
import sys
print(sys.version)
from keras.callbacks import ModelCheckpoint

# Numerical packages import
import scipy.io
import numpy as np


# mne imports
import mne
from mne import io
from mne.decoding import (
    CSP,
    GeneralizingEstimator,
    LinearModel,
    Scaler,
    SlidingEstimator,
    Vectorizer,
    cross_val_multiscore,
    get_coef,
)
from mne.datasets import sample
from autoreject import AutoReject, get_rejection_threshold
from eeg_positions import get_elec_coords, get_available_elec_names

# Other imports
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import os
import shutil

# Loading raw data from a specific subject and run
def load_raw_data(source_directory, subject_id, run_number):
    subject_directory = os.path.join(source_directory, f'S{subject_id:03d}')
    edf_file = os.path.join(subject_directory, f'S{subject_id:03d}R{run_number:02d}.edf')
    raw = mne.io.read_raw_edf(edf_file, preload=True)
    return raw

# Defining the annotations dictionary
annotations_dict = {
    '0': 'rest',
    '1': 'left fist',
    '2': 'right fist',
    '3': 'both fists',
    '4': 'feet',
    '5': 'left fist (visualizing)',
    '6': 'right fist (visualizing)',
    '7': 'both fists (visualizing)',
    '8': 'feet (visualizing)'
}

# Defining the alias dictionary
alias_dict = {
    'Fc5.': 'FC5', 'Fc3.': 'FC3', 'Fc1.': 'FC1', 'Fcz.': 'FCz', 'Fc2.': 'FC2', 'Fc4.': 'FC4', 'Fc6.': 'FC6',
    'C5..': 'C5', 'C3..': 'C3', 'C1..': 'C1', 'Cz..': 'Cz', 'C2..': 'C2', 'C4..': 'C4', 'C6..': 'C6',
    'Cp5.': 'CP5', 'Cp3.': 'CP3', 'Cp1.': 'CP1', 'Cpz.': 'CPz', 'Cp2.': 'CP2', 'Cp4.': 'CP4', 'Cp6.': 'CP6',
    'Fp1.': 'Fp1', 'Fpz.': 'Fpz', 'Fp2.': 'Fp2',
    'Af7.': 'AF7', 'Af3.': 'AF3', 'Afz.': 'AFz', 'Af4.': 'AF4', 'Af8.': 'AF8',
    'F7..': 'F7', 'F5..': 'F5', 'F3..': 'F3', 'F1..': 'F1', 'Fz..': 'Fz', 'F2..': 'F2', 'F4..': 'F4', 'F6..': 'F6', 'F8..': 'F8',
    'Ft7.': 'FT7', 'Ft8.': 'FT8',
    'T7..': 'T7', 'T8..': 'T8', 'T9..': 'T9', 'T10.': 'T10',
    'Tp7.': 'TP7', 'Tp8.': 'TP8',
    'P7..': 'P7', 'P5..': 'P5', 'P3..': 'P3', 'P1..': 'P1', 'Pz..': 'Pz', 'P2..': 'P2', 'P4..': 'P4', 'P6..': 'P6', 'P8..': 'P8',
    'Po7.': 'PO7', 'Po3.': 'PO3', 'Poz.': 'POz', 'Po4.': 'PO4', 'Po8.': 'PO8',
    'O1..': 'O1', 'Oz..': 'Oz', 'O2..': 'O2',
    'Iz..': 'Iz'
}

def modify_annotations(raw, round_number):
    """Modify annotations based on the round number."""
    # Create a copy of the annotations
    annotations = raw.annotations.copy()

    # Modify annotations based on the round number
    if round_number in [3, 7, 11]:
        for idx, ann in enumerate(annotations):
            if ann['description'] == 'T0':
                annotations.description[idx] = '0'
            elif ann['description'] == 'T1':
                annotations.description[idx] = '1'
            elif ann['description'] == 'T2':
                annotations.description[idx] = '2'
    elif round_number in [5, 9, 13]:
        for idx, ann in enumerate(annotations):
            if ann['description'] == 'T0':
                annotations.description[idx] = '0'
            elif ann['description'] == 'T1':
                annotations.description[idx] = '3'
            elif ann['description'] == 'T2':
                annotations.description[idx] = '4'
    elif round_number in [4, 8, 12]:
        for idx, ann in enumerate(annotations):
            if ann['description'] == 'T0':
                annotations.description[idx] = '0'
            elif ann['description'] == 'T1':
                annotations.description[idx] = '1'
            elif ann['description'] == 'T2':
                annotations.description[idx] = '2'
    elif round_number in [6, 10, 14]:
        for idx, ann in enumerate(annotations):
            if ann['description'] == 'T0':
                annotations.description[idx] = '0'
            elif ann['description'] == 'T1':
                annotations.description[idx] = '3'
            elif ann['description'] == 'T2':
                annotations.description[idx] = '4'

    # Set the modified annotations to the raw data
    raw.set_annotations(annotations)

import mne
import numpy as np
import matplotlib.pyplot as plt

# Set log level
mne.set_log_level('error')

# Definir o diretório fonte contendo as pastas dos sujeitos
source_directory = '/content/drive/MyDrive/NEUKO/data/BCICIV_2000/files'
#source_directory = './data/BCICIV_2000/files'

# Lista de canais de interesse
channels_of_interest = list(alias_dict.keys())

3.10.12 (main, Jul 29 2024, 16:56:48) [GCC 11.4.0]


In [ ]:
#X = []
#Y_concatenado = []
#X_concatenado = []
#epochs_list = []

col_names =  ['time', 'condition', 'epoch', 'Fc5.', 'Fc3.', 'Fc1.', 'Fcz.', 'Fc2.',
       'Fc4.', 'Fc6.', 'C5..', 'C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'C6..',
       'Cp5.', 'Cp3.', 'Cp1.', 'Cpz.', 'Cp2.', 'Cp4.', 'Cp6.', 'Fp1.', 'Fpz.',
       'Fp2.', 'Af7.', 'Af3.', 'Afz.', 'Af4.', 'Af8.', 'F7..', 'F5..', 'F3..',
       'F1..', 'Fz..', 'F2..', 'F4..', 'F6..', 'F8..', 'Ft7.', 'Ft8.', 'T7..',
       'T8..', 'T9..', 'T10.', 'Tp7.', 'Tp8.', 'P7..', 'P5..', 'P3..', 'P1..',
       'Pz..', 'P2..', 'P4..', 'P6..', 'P8..', 'Po7.', 'Po3.', 'Poz.', 'Po4.',
       'Po8.', 'O1..', 'Oz..', 'O2..', 'Iz..', 'volunteer']

my_df  = pd.DataFrame(columns = col_names)


for subject in range(81,91):
    for round in range(3, 15):
        if round not in [4, 8, 12, 6, 10, 14]:  # Pegando apenas os rounds sem evento feito, com visualization.
            raw = load_raw_data(source_directory, subject_id=subject, run_number=round)
            #raw.pick_channels(channels_of_interest)

            # Modificar as anotações com base no número do round
            modify_annotations(raw, round_number=round)

            # Pré-processamento dos dados
            notch_freq = 60  # Hz
            notch_width = 2  # Hz
            filt_raw = raw.resample(sfreq=128) # Resample the data
            filt_raw = filt_raw.copy().notch_filter(freqs=notch_freq, notch_widths=notch_width)
            filt_raw = filt_raw.copy().filter(2, 40, method='fir') # ver mais um pouco, tipo 1
            # filt_raw.plot(start=0, duration=8, scalings={'eeg': 1.5e-4})

            # Definir a montagem e os eventos
            filt_raw.info.set_montage('standard_1005', match_alias=alias_dict)
            unique_labels = np.unique(filt_raw.annotations.description)
            event_id = {label: idx for idx, label in enumerate(unique_labels)}
            events, event_id = mne.events_from_annotations(filt_raw, event_id=int)

            # Interpolação de canais ruins
            threshold = 0.4

            #channel_neighbors = compute_channel_adjacency(filt_raw)
            #bad_channels = find_bad_channels(filt_raw, threshold)
            #for channel in bad_channels:
            #    filt_raw.info['bads'].append(channel)
            #filt_raw.interpolate_bads(method='spline')

            # Normalização dos dados
            #data = filt_raw.get_data()
            #mean = np.mean(data, axis=1, keepdims=True)
            #std = np.std(data, axis=1, keepdims=True)
            #scaled_data = (data - mean) / std

            # Atualizar os dados normalizados de volta ao objeto Raw
            #filt_raw._data = scaled_data

            # Criar epochs
            epoch = mne.Epochs(filt_raw, events=events, tmin=0, tmax=4, proj=False, baseline=None, preload=True, verbose=False, picks='eeg')
            #epoch = epoch_completo.crop(tmin=0.35, tmax=0.55)

            #gui code
            df = epoch.to_data_frame()
            df['volunteer'] = subject

            my_df = pd.concat([my_df.reset_index(drop=True), df.reset_index(drop=True)], axis= 0)

#print(my_df)

<ipython-input-4-8fc193f51a35>:67: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  my_df = pd.concat([my_df.reset_index(drop=True), df.reset_index(drop=True)], axis= 0)


In [ ]:
my_df

,time,condition,epoch,Fc5.,Fc3.,Fc1.,Fcz.,Fc2.,Fc4.,Fc6.,...,Po7.,Po3.,Poz.,Po4.,Po8.,O1..,Oz..,O2..,Iz..,volunteer
0,0.000000,0,0,8.470329e-15,-3.388132e-15,6.776264e-15,3.388132e-15,1.185846e-14,1.016440e-14,6.776264e-15,...,8.470329e-16,5.929231e-15,-8.470329e-16,-5.082198e-15,-4.552802e-15,0.000000,-2.541099e-15,-1.270549e-15,-8.470329e-16,81
1,0.007812,0,0,1.068446e+01,1.329235e+01,1.642542e+01,1.386535e+01,1.480440e+01,1.205352e+01,9.014512e+00,...,9.861616e+00,1.017732e+01,8.240483e+00,7.222523e+00,5.083109e+00,6.308442,6.597830e+00,6.270722e+00,3.967503e+00,81
2,0.015625,0,0,2.385277e+01,2.753147e+01,3.037741e+01,2.456837e+01,2.498843e+01,2.022886e+01,1.524071e+01,...,8.277584e+00,1.012503e+01,9.957421e+00,8.190450e+00,2.604615e+00,3.977203,3.034577e+00,4.384879e+00,-3.163997e+00,81
3,0.023438,0,0,2.553633e+01,2.836146e+01,3.084740e+01,2.444655e+01,2.450505e+01,1.957079e+01,1.284951e+01,...,4.864818e+00,8.176537e+00,9.166269e+00,1.083979e+01,4.907019e+00,0.491918,-7.670168e-01,3.127565e+00,-7.864497e+00,81
4,0.031250,0,0,1.564657e+01,1.863546e+01,2.166790e+01,1.789813e+01,1.809288e+01,1.402551e+01,6.776957e+00,...,6.845194e+00,8.535020e+00,7.776006e+00,1.094530e+01,7.217256e+00,-1.368499,-3.456203e-01,2.761421e+00,-7.657514e+00,81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15385,3.968750,3,29,5.383728e+00,2.825930e+00,2.916586e+00,-7.780503e-01,8.247285e+00,1.767868e+01,6.354961e+00,...,-2.526893e+01,-5.293722e+01,-4.048556e+01,-1.741120e+01,-1.115504e+01,-32.026092,-2.920640e+01,-2.587878e+01,-1.497652e+01,90
15386,3.976562,3,29,3.165815e+01,1.406235e+01,2.317841e+01,1.733424e+01,2.664398e+01,1.921662e+01,4.855373e+01,...,-4.481319e+01,-5.763164e+01,-5.107585e+01,-3.100852e+01,-2.670592e+01,-52.444627,-4.324854e+01,-4.149310e+01,-3.889182e+01,90
15387,3.984375,3,29,1.199692e+01,1.169699e+01,3.169442e+01,2.242390e+01,3.605384e+01,2.729385e+01,7.458434e+01,...,-3.286830e+01,-3.162450e+01,-3.024040e+01,-1.962969e+01,-1.826386e+01,-35.895787,-2.908748e+01,-2.825249e+01,-3.540888e+01,90
15388,3.992188,3,29,2.545676e+01,4.054391e+01,4.847235e+01,3.290246e+01,4.034027e+01,3.853155e+01,5.948133e+01,...,2.263891e+01,2.075102e+01,1.881915e+01,1.095717e+01,7.607427e+00,14.021894,1.461998e+01,6.695813e+00,9.064982e+00,90


In [ ]:
my_df.epoch.unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43], dtype=object)

In [ ]:
data_frame_poc2 = my_df.copy()
data_frame_poc2["condition"] = data_frame_poc2["condition"].astype(int)
data_frame_poc2["epoch"] = data_frame_poc2["epoch"].astype(int)


data_frame_poc2.rename(columns=alias_dict, inplace=True)

data_frame_poc3 = data_frame_poc2.set_index("epoch")
data_frame_poc3 = data_frame_poc3.reset_index()

data_frame_poc3.head()

,epoch,time,condition,FC5,FC3,FC1,FCz,FC2,FC4,FC6,...,PO7,PO3,POz,PO4,PO8,O1,Oz,O2,Iz,volunteer
0,0,0.000000,0,8.470329e-15,-3.388132e-15,6.776264e-15,3.388132e-15,1.185846e-14,1.016440e-14,6.776264e-15,...,8.470329e-16,5.929231e-15,-8.470329e-16,-5.082198e-15,-4.552802e-15,0.000000,-2.541099e-15,-1.270549e-15,-8.470329e-16,81
1,0,0.007812,0,1.068446e+01,1.329235e+01,1.642542e+01,1.386535e+01,1.480440e+01,1.205352e+01,9.014512e+00,...,9.861616e+00,1.017732e+01,8.240483e+00,7.222523e+00,5.083109e+00,6.308442,6.597830e+00,6.270722e+00,3.967503e+00,81
2,0,0.015625,0,2.385277e+01,2.753147e+01,3.037741e+01,2.456837e+01,2.498843e+01,2.022886e+01,1.524071e+01,...,8.277584e+00,1.012503e+01,9.957421e+00,8.190450e+00,2.604615e+00,3.977203,3.034577e+00,4.384879e+00,-3.163997e+00,81
3,0,0.023438,0,2.553633e+01,2.836146e+01,3.084740e+01,2.444655e+01,2.450505e+01,1.957079e+01,1.284951e+01,...,4.864818e+00,8.176537e+00,9.166269e+00,1.083979e+01,4.907019e+00,0.491918,-7.670168e-01,3.127565e+00,-7.864497e+00,81
4,0,0.031250,0,1.564657e+01,1.863546e+01,2.166790e+01,1.789813e+01,1.809288e+01,1.402551e+01,6.776957e+00,...,6.845194e+00,8.535020e+00,7.776006e+00,1.094530e+01,7.217256e+00,-1.368499,-3.456203e-01,2.761421e+00,-7.657514e+00,81


In [ ]:
data_frame_poc3.condition.value_counts()


condition
0    477603
2    122607
3    118503
4    118503
1    117990
Name: count, dtype: int64

In [ ]:
data_frame_poc3.volunteer.unique()

array([81, 82, 83, 84, 85, 86, 87, 88, 89, 90], dtype=object)

In [ ]:
data_frame_poc3.volunteer.unique()

array([81, 82, 83, 84, 85, 86, 87, 88, 89, 90], dtype=object)

In [ ]:
df_all_persons_col_names =  ['Delta', 'Theta', 'Low_Alpha', 'High_Alpha', 'Low_Beta', 'Mid_Beta',
       'High_Beta', 'Gamma', 'EEG_condition', 'EEG_channel', 'EEG_epoch',
       'EEG_median_value', 'EEG_average_value', 'EEG_std_value', 'EEG_25_perc',
       'EEG_75_perc']

df_all_persons = pd.DataFrame(columns = df_all_persons_col_names)

for person in data_frame_poc3.volunteer.unique():
    for ch in data_frame_poc3.columns[3:66]:
        for cond in data_frame_poc3['condition'].unique():
            try:
                for epc in data_frame_poc3['epoch'].loc[(data_frame_poc3.condition == cond)].unique():
                    data_full = data_frame_poc3.loc[(data_frame_poc3.condition == cond) & (data_frame_poc3.epoch == epc)].reset_index()

                    data_full = data_full.reset_index()

                    data_full2 = data_full[['epoch', 'time', 'condition', ch,'volunteer']].copy()
                    data_full2[['epoch', 'time', 'condition', ch,'volunteer']]
                    column_index_to_rename = -2  # Index of the 'name' column
                    new_column_name = 'EEG_value'

                    data_full2.rename(columns={data_full2.columns[column_index_to_rename]: new_column_name}, inplace=True)

                    data = data_full2[["EEG_value"]].values

                    fft_vals = np.absolute(np.fft.rfft(data))

                    fft_freq = np.fft.rfftfreq(len(data), 1.0/160)
                    eeg_bands = {'Delta': (0, 4), #lethargic, not moving, not attentive
                             'Theta': (4, 8), #creative, intuitive; but may also be distracted, unfocused
                             'Low_Alpha': (8, 10), #inner-awareness of self, mind/body integration, balance
                             'High_Alpha': (10, 12), #centering, healing, mind/body connection
                             'Low_Beta': (12, 15), #relaxed yet focused, integrated
                             'Mid_Beta': (15, 18), #alert, active, but not agitated
                             'High_Beta': (18, 30), #mental activity, e.g. math, planning; alertness, agitation
                             'Gamma': (30, 45)} #mental activity, e.g. math, planning; alertness, agitation

                    eeg_band_fft = dict()

                    for band in eeg_bands:
                        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq <= eeg_bands[band][1]))[0]
                        eeg_band_fft[band] = np.mean(fft_vals[freq_ix])
                    df_bands = pd.DataFrame(columns=['band', 'val'])
                    df_bands['band'] = eeg_bands.keys()
                    df_bands['val'] = [eeg_band_fft[band] for band in eeg_bands]
                    df_bands_transp = df_bands.set_index('band').T.reset_index()
                    df_bands_transp['EEG_condition'] = cond
                    df_bands_transp['EEG_channel'] = ch
                    df_bands_transp['EEG_epoch'] = epc
                    df_bands_transp['EEG_person'] = person

                    df_bands_transp = df_bands_transp.drop('index', axis=1)
                    df_bands_transp = df_bands_transp.rename_axis(None, axis=1)

                    df_bands_transp['EEG_median_value'] = data_full2[["EEG_value"]].quantile(0.5).values
                    df_bands_transp['EEG_average_value'] = data_full2[["EEG_value"]].mean().values
                    df_bands_transp['EEG_std_value'] = data_full2[["EEG_value"]].std().values
                    df_bands_transp['EEG_25_perc'] = data_full2[["EEG_value"]].quantile(0.25).values
                    df_bands_transp['EEG_75_perc'] = data_full2[["EEG_value"]].quantile(0.75).values

                    df_all_persons = pd.concat([df_all_persons, df_bands_transp], axis=0).reset_index()
                    df_all_persons = df_all_persons.drop('index', axis=1)
            except:
                pass


<ipython-input-11-e4348bbd8127>:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all_persons = pd.concat([df_all_persons, df_bands_transp], axis=0).reset_index()


In [ ]:
df_all_persons

,Delta,Theta,Low_Alpha,High_Alpha,Low_Beta,Mid_Beta,High_Beta,Gamma,EEG_condition,EEG_channel,EEG_epoch,EEG_median_value,EEG_average_value,EEG_std_value,EEG_25_perc,EEG_75_perc,EEG_person
0,14.268197,17.723008,16.075484,12.175599,14.912630,18.190930,22.521170,31.764077,0,FC5,0,-1.898875,-0.515893,37.344661,-18.029825,14.104165,81.0
1,19.959198,17.202346,24.021498,24.861519,21.648079,15.396388,20.358888,33.549057,0,FC5,2,-2.391273,-0.133510,42.531225,-19.425141,15.002888,81.0
2,20.859749,18.893608,24.104068,16.078519,22.717498,11.742437,19.100309,32.054719,0,FC5,4,-2.281806,0.014082,39.193955,-18.641987,14.341310,81.0
3,18.903887,20.099763,23.053388,20.863765,17.097054,15.641095,19.600353,34.373755,0,FC5,6,-1.843053,0.008054,38.502566,-18.486577,15.091689,81.0
4,20.540943,15.323627,20.781158,20.440163,22.014327,16.484775,21.735521,33.851960,0,FC5,8,-1.959251,0.088703,40.768489,-19.896302,15.516408,81.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62365,11.138733,11.939045,13.437823,17.719542,24.613329,27.102334,18.877562,15.791901,4,O2,21,-0.924639,-0.279456,27.075450,-14.487677,12.582187,90.0
62366,17.003127,14.980647,14.931661,15.732734,17.198598,14.110337,17.684476,20.393651,4,O2,9,-0.789188,-0.122379,27.061557,-14.194289,12.037382,90.0
62367,17.105174,15.127965,37.780211,28.351769,40.508735,51.287712,24.575561,23.488750,4,O2,31,-2.352773,-0.178426,39.698296,-26.285287,25.347034,90.0
62368,33.710554,73.250656,27.607983,49.469373,27.173781,24.436368,79.085370,70.135981,4,O2,35,-2.231448,5.479702,64.081333,-30.791897,30.916004,90.0


In [ ]:
df_all_persons[['EEG_condition', 'EEG_channel', 'EEG_epoch']].loc[(df_all_persons.EEG_person == 5)]

,EEG_condition,EEG_channel,EEG_epoch


In [1]:
df_all_persons.to_csv('/content/drive/My Drive/NEUKO/outputs/df_all_persons_top_90.csv', index=False)

NameError: name 'df_all_persons' is not defined

In [ ]:
df_verify = pd.read_csv('/content/drive/My Drive/NEUKO/outputs/df_all_persons_top_90')
print(df_verify)

           Delta      Theta  Low_Alpha  High_Alpha   Low_Beta   Mid_Beta  \
0      14.268197  17.723008  16.075484   12.175599  14.912630  18.190930   
1      19.959198  17.202346  24.021498   24.861519  21.648079  15.396388   
2      20.859749  18.893608  24.104068   16.078519  22.717498  11.742437   
3      18.903887  20.099763  23.053388   20.863765  17.097054  15.641095   
4      20.540943  15.323627  20.781158   20.440163  22.014327  16.484775   
...          ...        ...        ...         ...        ...        ...   
62365  11.138733  11.939045  13.437823   17.719542  24.613329  27.102334   
62366  17.003127  14.980647  14.931661   15.732734  17.198598  14.110337   
62367  17.105174  15.127965  37.780211   28.351769  40.508735  51.287712   
62368  33.710554  73.250656  27.607983   49.469373  27.173781  24.436368   
62369  16.203457  29.101689  11.872935   22.844713  34.077462  66.142456   

       High_Beta      Gamma  EEG_condition EEG_channel  EEG_epoch  \
0      22.521170  